# Using Twitter Decahose with Cavium

## Table of Contents
  - [UM Hadoop Cavium Cluster](#um-hadoop-cavium-cluster)
    - [Setting Python Version](#setting-python-version)
  - [PySpark Interactive Shell](#pyspark-interactive-shell)
    - [Exit Interactive Shell](#exit-interactive-shell)
  - [Using Jupyter Notebook with PySpark](#using-jupyter-notebook-with-pyspark)
  - [Example Code](#example-code)
      - [Read in twitter file](#read-in-twitter-file)
      - [Selecting Data](#selecting-data)
        - [Getting Nested Data](#getting-nested-data)
        - [Getting Nested Data II](#getting-nested-data-ii)
      - [Summary](#summary)
      - [Saving Data](#saving-data)
      - [Complete Script](#complete-script)
      - [Example: Finding text in a Tweet](#text-in-tweet)

## UM Hadoop Cavium Cluster <a name='um-hadoop-cavium-cluster'>
Twitter data already resides in a directory on Cavium. Log in to Cavium to get started.

SSH to `cavium-thunderx.arc-ts.umich.edu` `Port 22` using a SSH client (e.g. PuTTY on Windows) and login using your Cavium account and two-factor authentication.

**Note:** ARC-TS has a [Getting Started with Hadoop User Guide](http://arc-ts.umich.edu/new-hadoop-user-guide/)

### Setting Python Version <a name='setting-python-version'>
Change Python version for PySpark to Python 3.X (instead of default Python 2.7) 

```
export PYSPARK_PYTHON=/bin/python3  
export PYSPARK_DRIVER_PYTHON=/bin/python3
```

## PySpark Interactive Shell <a name='pyspark-interactive-shell'>
The interactive shell is analogous to a python console. The following command starts up the interactive shell for PySpark with default settings in the `workshop` queue.  
`pyspark --master yarn --queue workshop`

The following line adds some custom settings.  The 'XXXX' should be a number between 4050 and 4099.  
`pyspark --master yarn --queue workshop --num-executors 500 --executor-memory 5g --conf spark.ui.port=XXXX`

**Note:** You might get a warning message that looks like `WARN Utils: Service 'SparkUI' could not bind on port 40XX. Attempting port 40YY.` This usually resolves itself after a few seconds. If not, try again at a later time.

The interactive shell does not start with a clean slate. It already has several objects defined for you. 
- `sc` is a SparkContext
- `sqlContext` is a SQLContext object
- `spark` is a SparkSession object

You can check this by typing the variable names.

### Exit Interactive Shell <a name='exit-interactive-shell'>
Type `exit()` or press Ctrl-D

## Using Jupyter Notebook with PySpark <a name='using-jupyter-notebook-with-pyspark'>
Currently, the Cavium configuration only supports Python 2.7 on Jupyter.

1. Open a command prompt/terminal in Windows/Mac. You should have putty in your PATH (for Windows).  Port 8889 is arbitrarily chosen.  
`putty.exe -ssh -L localhost:8889:localhost:8889 cavium-thunderx.arc-ts.umich.edu` (Windows)  
`ssh -L localhost:8889:localhost:8889 cavium-thunderx.arc-ts.umich.edu` (Mac/Linux)
2. This should open a ssh client for Cavium. Log in as usual.
3. From the Cavium terminal, type the following (replace XXXX with number between 4050 and 4099):

`export PYSPARK_PYTHON=/bin/python3  # not functional code`  
`export PYSPARK_DRIVER_PYTHON=jupyter`  
`export PYSPARK_DRIVER_PYTHON_OPTS='notebook --no-browser --port=8889'`  
`pyspark --master yarn --queue workshop --num-executors 500 --executor-memory 5g --conf spark.ui.port=XXXX`
```
4. Copy/paste the URL (from your terminal where you launched jupyter notebook) into your browser. The URL should look something like this but with a different token.
http://localhost:8889/?token=745f8234f6d0cf3b362404ba32ec7026cb6e5ea7cc960856
5. You should be connected.

In [1]:
sc

<SparkContext master=yarn appName=PySparkShell>

In [2]:
sqlContext

In [3]:
spark

Check Python version

In [32]:
import sys
sys.version

'2.7.5 (default, Oct 31 2018, 18:48:32) \n[GCC 4.8.5 20150623 (Red Hat 4.8.5-36)]'

## Example Code <a name='example-code'>
Generic PySpark data wrangling commands can be found at https://github.com/caocscar/workshops/blob/master/pyspark/pyspark.md

## Read in twitter file <a name='read-in-twitter-file'>
The twitter data is stored in JSONLINES format and compressed using bz2. PySpark has a `sqlContext.read.json` function that can handle this for us (including the decompression).

In [5]:
import os
wdir = '/var/twitter/decahose/raw'
df = sqlContext.read.json(os.path.join(wdir,'decahose.2018-03-02.p2.bz2'))

This reads the JSONLINES data into a PySpark DataFrame. We can see the structure of the JSON data using the `printSchema` method.

In [7]:
df.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- embeddable: boolean (nullable = true)
 |    |    |    |    |-- monetizable: bo

The schema shows the "root-level" attributes as columns of the dataframe. Any nested data is squashed into arrays of values (no keys included).

**Reference**
 - PySpark JSON Files Guide https://spark.apache.org/docs/latest/sql-data-sources-json.html

 - Twitter Tweet Objects https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html

## Selecting Data <a name='selecting-data'>
For example, if we wanted to see what the tweet text is and when it was created, we could do the following.

In [8]:
tweet = df.select('created_at','text')
tweet.printSchema()
tweet.show(5)

root
 |-- created_at: string (nullable = true)
 |-- text: string (nullable = true)

+--------------------+--------------------+
|          created_at|                text|
+--------------------+--------------------+
|Sat Mar 03 04:57:...|RT @nyorai_fgo: ア...|
|Sat Mar 03 04:57:...|絶対サンダイオー出るのやばいんだが...|
|Sat Mar 03 04:57:...|come hang out whi...|
|Sat Mar 03 04:57:...|RT @minstarcholee...|
|Sat Mar 03 04:57:...|RT @prswunews: 🙏...|
+--------------------+--------------------+
only showing top 5 rows



The output is truncated by default. We can override this using the truncate argument.

In [9]:
tweet.show(5, truncate=False)

+------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------+
|created_at                    |text                                                                                                                                            |
+------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------+
|Sat Mar 03 04:57:18 +0000 2018|RT @nyorai_fgo: アゲハ蝶で推しカプに涙するオタクは
「貴方に会えたそれだけで良かった世界に光が満ちた」系
「愛されたいと願ってしまった世界が表情を変えた」系
「貴方が望むのならこの身などいつでも差し出していい」系
「ラーラーラーラーーーラーーーーラーーーー（語彙…    |
|Sat Mar 03 04:57:18 +0000 2018|絶対サンダイオー出るのやばいんだがサッヴァーク当たったから良いものの                                                                                                              |
|Sat Mar 03 04:57:18 +0000 2018|come hang out while I strim some #Destiny https://t.co/l2ntwt5GT3             

### Getting Nested Data <a name='getting-nested-data'>
What if we wanted to get at data that was nested? Like in `user`.

In [11]:
user = df.select('user')
user.printSchema()
user.show(1, truncate=False)

root
 |-- user: struct (nullable = true)
 |    |-- contributors_enabled: boolean (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- default_profile: boolean (nullable = true)
 |    |-- default_profile_image: boolean (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- favourites_count: long (nullable = true)
 |    |-- follow_request_sent: string (nullable = true)
 |    |-- followers_count: long (nullable = true)
 |    |-- following: string (nullable = true)
 |    |-- friends_count: long (nullable = true)
 |    |-- geo_enabled: boolean (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- id_str: string (nullable = true)
 |    |-- is_translator: boolean (nullable = true)
 |    |-- lang: string (nullable = true)
 |    |-- listed_count: long (nullable = true)
 |    |-- location: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- notifications: string (nullable = true)
 |    |-- profile_background_color: str

This returns a single column `user` with the nested data in a list (technically a `struct`).

We can select nested data using the `.` notation.

In [12]:
names = df.select('user.name','user.screen_name')
names.printSchema()
names.show(5)

root
 |-- name: string (nullable = true)
 |-- screen_name: string (nullable = true)

+----------+-----------+
|      name|screen_name|
+----------+-----------+
|        夜雲| ya_kumo229|
|      ぽけねこ|pokeneko867|
|Big Fletch|BigFletchWC|
|    m.byul|    mbyul_m|
|    DEAR12| Dearbabo12|
+----------+-----------+
only showing top 5 rows



To expand ALL the data into individual columns, you can use the `.*` notation.

In [13]:
allcolumns = df.select('user.*')
allcolumns.printSchema()
allcolumns.show(4)

root
 |-- contributors_enabled: boolean (nullable = true)
 |-- created_at: string (nullable = true)
 |-- default_profile: boolean (nullable = true)
 |-- default_profile_image: boolean (nullable = true)
 |-- description: string (nullable = true)
 |-- favourites_count: long (nullable = true)
 |-- follow_request_sent: string (nullable = true)
 |-- followers_count: long (nullable = true)
 |-- following: string (nullable = true)
 |-- friends_count: long (nullable = true)
 |-- geo_enabled: boolean (nullable = true)
 |-- id: long (nullable = true)
 |-- id_str: string (nullable = true)
 |-- is_translator: boolean (nullable = true)
 |-- lang: string (nullable = true)
 |-- listed_count: long (nullable = true)
 |-- location: string (nullable = true)
 |-- name: string (nullable = true)
 |-- notifications: string (nullable = true)
 |-- profile_background_color: string (nullable = true)
 |-- profile_background_image_url: string (nullable = true)
 |-- profile_background_image_url_https: string (nulla

Some nested data is stored in an `array` instead of `struct`.

In [14]:
arr = df.select('entities.user_mentions.name')
arr.printSchema()
arr.show(5)

root
 |-- name: array (nullable = true)
 |    |-- element: string (containsNull = true)

+--------------------+
|                name|
+--------------------+
|  [〆切から逃げるな3/18ア34b]|
|                  []|
|                  []|
|[🌙 #เกียมต้อนรับ...|
|               [มศว]|
+--------------------+
only showing top 5 rows



The data is stored in an `array` similar as before. We can use the `explode` function to extract the data from an `array`.

In [15]:
from pyspark.sql.functions import explode

arr2 = df.select(explode('entities.user_mentions.name'))
arr2.printSchema()
arr2.show(5)

root
 |-- col: string (nullable = true)

+-------------------+
|                col|
+-------------------+
|   〆切から逃げるな3/18ア34b|
|🌙 #เกียมต้อนรับผัว|
|                มศว|
| Dj Tannie Swiss 🎧|
|  paidamoyo marimbe|
+-------------------+
only showing top 5 rows



If we wanted multiple columns under user_mentions, we'd be tempted to use multiple `explode` statements as so.

In [16]:
df.select(explode('entities.user_mentions.name'), explode('entities.user_mentions.screen_name'))

AnalysisException: u'Only one generator allowed per select clause but found 2: explode(entities.user_mentions.name AS `name`), explode(entities.user_mentions.screen_name AS `screen_name`);'

This generates an error: *Only one generator allowed per select clause but found 2:*

We can get around this by using `explode` on the top most key with an `alias` and then selecting the columns of interest.

In [17]:
mentions = df.select(explode('entities.user_mentions').alias('user_mentions'))
mentions.printSchema()
mentions2 = mentions.select('user_mentions.name','user_mentions.screen_name')
mentions2.show(5)

root
 |-- user_mentions: struct (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- id_str: string (nullable = true)
 |    |-- indices: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- name: string (nullable = true)
 |    |-- screen_name: string (nullable = true)

+-------------------+-------------+
|               name|  screen_name|
+-------------------+-------------+
|   〆切から逃げるな3/18ア34b|   nyorai_fgo|
|🌙 #เกียมต้อนรับผัว|minstarcholee|
|                มศว|    prswunews|
| Dj Tannie Swiss 🎧|   tmarimbe23|
|  paidamoyo marimbe|   PaidamoyoM|
+-------------------+-------------+
only showing top 5 rows



### Getting Nested Data II <a name='getting-nested-data-ii'>
What if we wanted to get at data in a list? Like the indices in `user_mentions`.

In [18]:
idx = mentions.select('user_mentions.indices')
idx.printSchema()
idx.show(5)

root
 |-- indices: array (nullable = true)
 |    |-- element: long (containsNull = true)

+--------+
| indices|
+--------+
| [3, 14]|
| [3, 17]|
| [3, 13]|
| [5, 16]|
|[22, 33]|
+--------+
only showing top 5 rows



The schema shows that the data is in an `array` type. For some reason, `explode` will put each element in its own row. Instead, we can use the `withColumn` method to index the list elements.

In [35]:
idx2 = idx.withColumn('first', idx['indices'][0]).withColumn('second', idx['indices'][1])
idx2.show(5)

+--------+-----+------+
| indices|first|second|
+--------+-----+------+
| [3, 14]|    3|    14|
| [3, 17]|    3|    17|
| [3, 13]|    3|    13|
| [5, 16]|    5|    16|
|[22, 33]|   22|    33|
+--------+-----+------+
only showing top 5 rows



Why the difference?  Because the underlying element is not a `struct` data type but a `long` instead.

## Summary <a name='summary'>
So if you access JSON data in Python like this:

`(tweet['created_at'], tweet['user']['name'], tweet['user']['screen_name'], tweet['text'])`

The equivalent of a PySpark Dataframe would be like this:
`df.select('created_at','user.name','user.screen_name','text')`

## Saving Data <a name='saving-data'>
Once you have constructed your PySpark DataFrame of interest, you should save it (append or overwrite) as a parquet file as so.

In [22]:
folder = 'twitterExtract'
df.write.mode('overwrite').parquet(folder)

## Complete Script <a name='complete-script'>
Here is a sample script which combines everything we just covered. It extracts a four column DataFrame.

In [23]:
import os
from pyspark.sql.functions import explode

wdir = '/var/twitter/decahose/raw'
file = 'decahose.2018-03-02.p2.bz2'
df = sqlContext.read.json(os.path.join(wdir,file))
four = df.select('created_at','user.name','user.screen_name','text')
folder = 'twitterExtract'
four.write.mode('overwrite').parquet(folder)

## Example: Finding text in a Tweet <a name='text-in-tweet'>
Read in parquet file.

In [24]:
folder = 'twitterDemo'
df = sqlContext.read.parquet(folder)

Below are several ways to match text
***

Exact match `==`

In [25]:
hello = df.filter(df.text == 'hello world')
hello.show(10)

+--------------------+---------------+--------------+-----------+
|          created_at|           name|   screen_name|       text|
+--------------------+---------------+--------------+-----------+
|Wed Jul 03 10:10:...|         shefty|shefty05026540|hello world|
|Tue Jul 02 14:46:...|     Fathur2911|    fathur2911|hello world|
|Fri Jul 05 14:47:...|keru robot mode|          keru|hello world|
|Tue Jul 02 05:42:...|        balmunc|      balmunc1|hello world|
|Wed Jul 03 04:53:...|  fanax20082006| fanax20082006|hello world|
|Mon Jul 01 05:44:...|           Leah|   leahjames24|hello world|
|Wed Jul 03 02:29:...|           Niño|      NyouNyii|hello world|
|Fri Jul 05 01:19:...|        やっぱり甲子園|        hsbbjp|hello world|
|Thu Jul 04 15:51:...|           ささけん|        KRPK_A|hello world|
|Wed Jul 03 02:57:...|     spirit.wan|     spiritwan|hello world|
+--------------------+---------------+--------------+-----------+
only showing top 10 rows



`contains` method

In [26]:
food = df.filter(df['text'].contains(' food'))
food = food.select('text')
food.show(10, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                                       |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
|RT @JIMINSPROMlSE: ‘focuses on her mouth’ it’s an advertisement for food what else is she supposed to do with it ?? play with it like a bar…               |
|RT @xesixt: bule di comment section street food ini bawel ya, “wear some fucking glove you fuck” bawel lu londo cebok kok pake tisu hhh                    |
|RT @nayelly_nails: Even if I get my food first, I will wait for you to get yours to start eating https://t.co/9zAlhknJXP                                   |
|i thoroughly examine my food alittle more extra now

`startswith` method

In [27]:
once = df.filter(df.text.startswith('Once'))
once = once.select('text')
once.show(10, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                        |
+--------------------------------------------------------------------------------------------------------------------------------------------+
|Once zaleel always zaleel 
#fakharZaleel                                                                                                    |
|Once again my illiterate friends we dont take our husbands’ names here in Egypt, there’s no Mr and Mrs X, are you t… https://t.co/cYQ1yEJ6T6|
|Once you get to know about the particular hazards that occur at your workplace, then it will help you in reducing t… https://t.co/STDgBsX9Nu|
|Once you find ways to happily achieve what you want with what you have, then when you have abundance you know you're flying in colors. 😁   |


`endswith` method

In [28]:
ming = df.filter(df['text'].endswith('ming'))
ming = ming.select('text')
ming.show(10, truncate=False)

+----------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                  |
+----------------------------------------------------------------------------------------------------------------------+
|#StrangerThings3Premiere was underwhelming                                                                            |
|@GouldyGaming Love to see you videos 😊@GouldyGaming                                                                  |
|RT @vanjess: Divine timing                                                                                            |
|The amount of layers people in Paris wear in 90-degree weather is alarming                                            |
|https://t.co/t56KLqJxXo

I don’t know if you saw this, but yes, this, yes !!!!!
@idreamofcumming                      |
|I’m not 5sos but when is a new a

`like` method using SQL wildcards

In [29]:
mom = df.filter(df.text.like('%mom_'))
mom = mom.select('text')
mom.show(10, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                        |
+--------------------------------------------------------------------------------------------------------------------------------------------+
|【定期】ブログやってますのでよかったら来てやって下さい(*´∀｀)行ったイベントのレポ/ITなお話/気になる小ネタ/思うことなどなど頑張って書きます(*´ω｀*)b https://t.co/CZsRQcdsqf #mosmome                         |
|@iazs97 سناباتك تفتح النفس https://t.co/0QpEdLmomV                                                                                          |
|@ghsolowonda @johndumelo1 No please I'm on momo                                                                                             |
|I'm scared that the tickets would go on sale while I'll be at the camp.. also I have to ask my mom😭                                        |


regular expressions ([workshop material](https://github.com/caocscar/workshops/tree/master/regex))

In [34]:
regex = df.filter(df['text'].rlike('[ia ]king'))
regex = regex.select('text')
regex.show(10, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                        |
+--------------------------------------------------------------------------------------------------------------------------------------------+
|At least there wre some trees around. Love the light making circular patterns on branches in the winter. https://t.co/8niWN7vevH            |
|#HanggangMayMayWard  lalaban ako  at susuportahan ko sila sa abot ng aking makakaya                                                         |
|RT @VisitGraceland: Thank you, thank you very much to @ArgoMemphis for the incredible #Elvis-themed fireworks. The king would’ve loved this…|
|RT @Kevinfischer593: Here's a video of me taking ice cream and putting it in my cart and not licking it https://t.co/t5sq9al0DN             |

Applying more than one condition. When building DataFrame boolean expressions, use
- `&` for `and`
- `|` for `or`
- `~` for `not`  

In [31]:
resta = df.filter(df.text.contains('resta') & df.text.endswith('ing'))
resta = resta.select('text')
resta.show(10, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                        |
+--------------------------------------------------------------------------------------------------------------------------------------------+
|Le Relais, un restaurant fertile - L'Hôtellerie Restauration https://t.co/suSRDWyBeU #paris #coworking                                      |
|RT @Amrut58731711: @Indiamining @PMOIndia @nstomar @nitin_gadkari @goacm @makeinindia #GoaMining #restartGoamining                          |
|@sam_carrington1 A birthday meal at a seafood restaurant I had a bad reaction to mussels, spent the night vomiting                          |
|RT @SHRIKRISHNA8484: @Indiamining Restart Goa mining operation
#200days  #restartgoamining  #goamining                                      |

**Reference**: http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column